In [109]:
import pandas as pd
import json
from newspaper import Article

In [123]:
pd.set_option("display.max_rows", None, "display.max_columns", None)
pd.set_option('display.max_colwidth', None)

In [31]:
#!pip install spacy

In [47]:
chat = pd.read_csv('./sample_chat.csv')

In [49]:
chat

,Thank you for your patience. I see you have an active subscription for Creative Cloud All Apps with 100GB EDU on an Annual Contract / Monthly Payment plan on the account. Its great news to hear that you are getting the subscription from work!!Congratulations Husam. I will cancel the subscription and waive off the cancellation charges as I reckon it is unnecessary to pay when you already get it from work.


In [37]:
url = 'https://www.cnbc.com/2022/07/27/fed-decision-july-2022-.html'

In [38]:
article = Article(url)

In [39]:
article.download()

In [40]:
article.parse()

In [46]:
article.text

'The Federal Reserve on Wednesday enacted its second consecutive 0.75 percentage point interest rate increase as it seeks to tamp down runaway inflation without creating a recession.\n\n\n\nIn taking the benchmark overnight borrowing rate up to a range of 2.25%-2.5%, the moves in June and July represent the most stringent consecutive action since the Fed began using the overnight funds rate as the principal tool of monetary policy in the early 1990s. While the fed funds rate most directly impacts what banks charge each other for short-term loans, it feeds into a multitude of consumer products such as adjustable mortgages, auto loans and credit cards. The increase takes the funds rate to its highest level since December 2018.\n\n\n\nMarkets largely expected the move after Fed officials telegraphed the increase in a series of statements since the June meeting. Stocks hit their highs after Fed Chair Jerome Powell left the door open about its next move at the September meeting, saying it w

In [110]:
with open('./chat_transcript.json') as data_file:    
    json_data = json.load(data_file)
    
df = pd.json_normalize(json_data['Transcript'])
text = df['Content'].to_string(index=False)
text

'                                               NaN\n      [customer connecting with agent - disregard]\n                                             agent\nHi, I\'m the Adobe Virtual Assistant. How can I ...\n{"genericAttachments":[{"title":"I want to make...\n                                  Continue in chat\n{"genericAttachments":[{"title":"Ok. Let me con...\n                                 Manage my account\n      Ok. Connecting you to an agent who can help.\nConnecting you with a senior specialist as soon...\n                                               NaN\nCUSTOM_SYSTEM_MESSAGE_You are now connected to ...\n                            email-Tpjpr9@adobe.com\nHello, I\'d be happy to help you today, let me f...\n                                       Hi Aprajita\nI want to know which subscription is active on ...\nSure Tp Train, may i know u want to know Subscr...\n                              for tpjpr9@adobe.com\nthankyou for confirming TP Training, as i can c...\n         

In [117]:
text

'                                               NaN\n      [customer connecting with agent - disregard]\n                                             agent\nHi, I\'m the Adobe Virtual Assistant. How can I ...\n{"genericAttachments":[{"title":"I want to make...\n                                  Continue in chat\n{"genericAttachments":[{"title":"Ok. Let me con...\n                                 Manage my account\n      Ok. Connecting you to an agent who can help.\nConnecting you with a senior specialist as soon...\n                                               NaN\nCUSTOM_SYSTEM_MESSAGE_You are now connected to ...\n                            email-Tpjpr9@adobe.com\nHello, I\'d be happy to help you today, let me f...\n                                       Hi Aprajita\nI want to know which subscription is active on ...\nSure Tp Train, may i know u want to know Subscr...\n                              for tpjpr9@adobe.com\nthankyou for confirming TP Training, as i can c...\n         

In [104]:
text="""In light of recent news from Apple, facial recognition technology is now the talk of the town. Ranging from privacy concerns to a curious boyfriend unlocking his girlfriend’s phone while she sleeps. The first commercial application of 3D facial recognition means that biometric technology is advancing at a staggering rate but so is the ability to hack or fool facial recognition tech.

Today, 99% of the infrastructure scattered around the world consists of 2D cameras capable of running advanced facial recognition software and it will likely be years before a physical overhaul to 3D cameras takes place. Hackers, hobbyists, educators and the like have been developing creative ways to either fool or hide from 2D face recognition in a perpetually innovative game of cat and mouse.

The introduction of 3D recognition will give way to an entirely new thought process of how to beat an incredibly advanced technology, and also raise some questions as to how and by whom it will be used. The fight is futile as the effort exerted to trick this technology is better spent working with the facial recognition companies to enforce acceptable regulations.

First, I question the intention of these types of face recognition-fooling solutions. Are they for bad actors hoping to sneak away undetected? Are they for people who are concerned about their picture being stored in a massive database (I’d worry more about posting a picture to Facebook…)? Are they to demonstrate the inherent risk of biometric information?

The variations that humans have designed to deceive the machine are more likely to make you stand out than blend in. Such as custom makeup, hairstyles, masks, a pair of glasses designed at Carnegie Melon consisting of camouflage patterns or simply wearing a box over your head. Some of them are realistic, but most are just for fun.

As these techniques improve, so does the technology capable of identifying and training against them resulting in more robust face recognition software. These solutions seem like a great way to protect individuals who are looking to keep their identity hidden for nefarious reasons.

A further concern would be detecting an attempt to impersonate an individual by using a picture of their face. With 3D cameras, this type of hack is very difficult to achieve due to the complexity of depth measurements. Facial recognition technology can run liveness detection or use additional hardware for heat sensing, both of which are capable of being tricked to a degree as seen with Samsung and Google.

The availability of public facial images is far more concerning due to the ease of which you would be able to render a 3D model from a small set of images and use virtual reality tools to project a face.

On the surface, these types of hacks would seem alarming but if gaining access to your phone now is as easy as guessing a 4-digit passcode, society should deem these tolerable risks.

Ultimately it is on facial recognition companies and Trueface to mitigate this risk by being more innovative, detecting consistencies in attacks and training the technology to identify when hacks occur. This game will continue to be played and become significantly more sophisticated with the adoption of 3D facial recognition.

"""

In [111]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation
from heapq import nlargest

In [112]:
def summarize(text, per):
    nlp = spacy.load('en_core_web_sm')
    doc= nlp(text)
    tokens=[token.text for token in doc]
    word_frequencies={}
    for word in doc:
        if word.text.lower() not in list(STOP_WORDS):
            if word.text.lower() not in punctuation:
                if word.text not in word_frequencies.keys():
                    word_frequencies[word.text] = 1
                else:
                    word_frequencies[word.text] += 1
    max_frequency=max(word_frequencies.values())
    for word in word_frequencies.keys():
        word_frequencies[word]=word_frequencies[word]/max_frequency
    sentence_tokens= [sent for sent in doc.sents]
    sentence_scores = {}
    for sent in sentence_tokens:
        for word in sent:
            if word.text.lower() in word_frequencies.keys():
                if sent not in sentence_scores.keys():                            
                    sentence_scores[sent]=word_frequencies[word.text.lower()]
                else:
                    sentence_scores[sent]+=word_frequencies[word.text.lower()]
    select_length=int(len(sentence_tokens)*per)
    summary=nlargest(select_length, sentence_scores,key=sentence_scores.get)
    final_summary=[word.text for word in summary]
    summary=''.join(final_summary)
    return summary

In [113]:
summarize(text, 0.05)

"Connecting you with a senior specialist as soon...\n                                               NaN\nCUSTOM_SYSTEM_MESSAGE_You are now connected to ...\n                            email-Tpjpr9@adobe.com\nHello, I'd be happy to help you today, let me f..."

In [20]:
python -m spacy download en_core_web_sm

SyntaxError: invalid syntax (581980377.py, line 1)

In [21]:
nlp = spacy.load('en_core_web_sm')

In [63]:
from gensim.summarization import summarize
import wikipedia
import re

ModuleNotFoundError: No module named 'gensim.summarization'

In [66]:
!pip install gensim==3.8.3

     |████████████████████████████████| 23.4 MB 2.4 MB/s eta 0:00:01
  Created wheel for gensim: filename=gensim-3.8.3-cp39-cp39-macosx_10_9_x86_64.whl size=24173846 sha256=c3f2ae84e42b39f02e40e49736e3221a51b95aa49974f1fd2a8758770eca9e78
  Stored in directory: /Users/tek88627/Library/Caches/pip/wheels/ca/5d/af/618594ec2f28608c1d6ee7d2b7e95a3e9b06551e3b80a491d6
Successfully built gensim
  Attempting uninstall: gensim
    Found existing installation: gensim 4.2.0
    Uninstalling gensim-4.2.0:
      Successfully uninstalled gensim-4.2.0


In [61]:
file_url_after_twenty=r'./chat.txt'
with open(file_url_after_twenty, 'r') as f:
        contents = f.read()

In [62]:
contents

'\ufeffThe Project Gutenberg EBook of North of Boston, by Robert Frost\n\nThis eBook is for the use of anyone anywhere at no cost and with\nalmost no restrictions whatsoever.  You may copy it, give it away or\nre-use it under the terms of the Project Gutenberg License included\nwith this eBook or online at www.gutenberg.org\n\n\nTitle: North of Boston\n\nAuthor: Robert Frost\n\nPosting Date: February 15, 2009 [EBook #3026]\nRelease Date: January, 2002\n\nLanguage: English\n\n\n*** START OF THIS PROJECT GUTENBERG EBOOK NORTH OF BOSTON ***\n\n\n\n\nProduced by David Reed\n\n\n\n\n\nNORTH OF BOSTON\n\nBy Robert Frost\n\n\n\n\n          TO\n          E. M. F.\n          THIS BOOK OF PEOPLE\n\n\n\n\nTHE PASTURE\n\n        I\'M going out to clean the pasture spring;\n        I\'ll only stop to rake the leaves away\n        (And wait to watch the water clear, I may):\n        I sha\'n\'t be gone long.--You come too.\n        I\'m going out to fetch the little calf\n        That\'s standing by

In [9]:
pd.set_option("display.max_rows", None, "display.max_columns", None)
pd.set_option('display.max_colwidth', None)

In [13]:
import pandas as pd
import json
with open('./chat_transcript1.json') as data_file:    
    json_data = json.load(data_file)
    
df = pd.json_normalize(json_data['Transcript'])
#df
text = df[['ContentType','DisplayName','Content','ParticipantRole','Type']]#.to_string(index=False)
text

,ContentType,DisplayName,Content,ParticipantRole,Type
0,application/vnd.amazonaws.connect.event.participant.joined,TP Training,NaN,CUSTOMER,EVENT
1,text/plain,TP Training,[customer connecting with agent - disregard],CUSTOMER,MESSAGE
2,text/plain,TP Training,agent,CUSTOMER,MESSAGE
3,text/plain,BOT,"Hi, I'm the Adobe Virtual Assistant. How can I help you?",SYSTEM,MESSAGE
4,text/plain,BOT,"{""genericAttachments"":[{""title"":""I want to make sure I understand clearly. How would you like to connect with an agent?"",""buttons"":[{""text"":""On the phone"",""value"":""On the phone"",""click"":{""actions"":[{""text"":""On the phone"",""type"":""publishText""}]}},{""text"":""Continue in chat"",""value"":""Continue in chat"",""click"":{""actions"":[{""text"":""Continue in chat"",""type"":""publishText""}]}}]}]}",SYSTEM,MESSAGE
5,text/plain,TP Training,Continue in chat,CUSTOMER,MESSAGE
6,text/plain,BOT,"{""genericAttachments"":[{""title"":""Ok. Let me connect you to the right agent. Which of these categories most closely describes your issue?"",""buttons"":[{""text"":""Try or buy an app"",""value"":""Try or buy an app"",""click"":{""actions"":[{""text"":""Try or buy an app"",""type"":""publishText""}]}},{""text"":""Solve a billing or payment issue"",""value"":""Solve a billing or payment issue"",""click"":{""actions"":[{""text"":""Solve a billing or payment issue"",""type"":""publishText""}]}},{""text"":""Cancel a plan"",""value"":""Cancel a plan"",""click"":{""actions"":[{""text"":""Cancel a plan"",""type"":""publishText""}]}},{""text"":""Manage my account"",""value"":""Manage my account"",""click"":{""actions"":[{""text"":""Manage my account"",""type"":""publishText""}]}},{""text"":""Troubleshoot a product issue"",""value"":""Troubleshoot a product issue"",""click"":{""actions"":[{""text"":""Troubleshoot a product issue"",""type"":""publishText""}]}}]}]}",SYSTEM,MESSAGE
7,text/plain,TP Training,Manage my account,CUSTOMER,MESSAGE
8,text/plain,BOT,Ok. Connecting you to an agent who can help.,SYSTEM,MESSAGE
9,text/plain,SYSTEM_MESSAGE,Connecting you with a senior specialist as soon as possible.,SYSTEM,MESSAGE


In [20]:
import pandas as pd
import json

pd.set_option("display.max_rows", None, "display.max_columns", None, 'display.max_colwidth', None)

with open('./chat_transcript1.json') as data_file:    
    json_data = json.load(data_file)
    
df = pd.json_normalize(json_data['Transcript'])

base_string = "{name} : {message}"

df['chat'] = [base_string.format(name=x,message=y) for x,y in zip(df['DisplayName'], df['Content'])]

chat_summary = df['chat'].to_string(index=False)
print(chat_summary)

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    TP Training : nan
                                                                                                                                  

In [21]:
import pandas as pd
import json

pd.set_option("display.max_rows", None, "display.max_columns", None, 'display.max_colwidth', None)

with open('./chat_transcript5.json') as data_file:    
    json_data = json.load(data_file)
print(json_data)
    
df = pd.json_normalize(json_data['Transcript'])

base_string = "{name} : {message}"

df['chat'] = [base_string.format(name=x,message=y) for x,y in zip(df['DisplayName'], df['Content'])]

chat_summary = df['chat'].to_string(index=False)
print(chat_summary)

{'Version': '2019-08-26', 'AWSAccountId': '831530512671', 'InstanceId': '853e1479-e0a6-4d0a-aa9d-cf1434d738da', 'InitialContactId': '03cf290e-b46e-45cb-96ff-5247e7f68a7a', 'ContactId': '03cf290e-b46e-45cb-96ff-5247e7f68a7a', 'Participants': [{'ParticipantId': '4450fd3b-80f5-40fe-96fb-0d4463c43101'}, {'ParticipantId': '9a656c0a-d4ed-446d-b057-25116951ff51'}, {'ParticipantId': 'aad08b72-9a5b-4b0c-ba38-791648a91fec'}], 'Transcript': [{'ContentType': 'application/vnd.amazonaws.connect.event.participant.joined', 'Id': '360676de-4a41-43e8-9f23-852c36a5dabd', 'DisplayName': 'TP Training', 'Type': 'EVENT', 'ParticipantRole': 'CUSTOMER', 'AbsoluteTime': '2022-08-02T14:57:22.874Z', 'ParticipantId': '9a656c0a-d4ed-446d-b057-25116951ff51'}, {'ContentType': 'text/plain', 'Content': '[customer connecting with agent - disregard]', 'Id': 'faf24adf-2014-42a2-895b-8d242a940a6e', 'DisplayName': 'TP Training', 'Type': 'MESSAGE', 'ParticipantRole': 'CUSTOMER', 'AbsoluteTime': '2022-08-02T14:57:27.725Z', 'P